In [ ]:
import pandas as pd
import numpy as np

## Import Dataframes

In [ ]:
#redacted

### Liquid Host Range Analysis

In [ ]:
#user inputs
AUC_hit_cutoff = 1 #redacted
strain_growth_threshold = 1 #redacted

#### LB Average Replicates

In [ ]:
#Average the blank strain AUC and endpoint OD
LB_blanks_df = LHR_LB[LHR_LB["Phage"] == "blank_1"]
LB_strain_avg = LB_blanks_df.groupby('Strain')["Control AUC", "Control Endpoint OD"].agg(['mean', np.std, 'count']).reset_index()
LB_strain_avg.columns = ["Strain", "Control AUC", "Control AUC StDev", "AUC count", "Control Endpoint OD", "Control OD StDev", "Control Replicates"]
LB_strain_avg = LB_strain_avg.drop(columns=["AUC count"])
                 
# #get which strains didn't reach endpoint OD threshold (usually #redacted)
LB_strains_no_grow = LB_blanks_df[(LB_blanks_df["Strain"] != "LB") & (LB_blanks_df['Control Endpoint OD'] < strain_growth_threshold)]

# #substitute in average control AUCs and endpoint ODs for all strains
LB_ctrl_merged_df = pd.merge(LHR_LB, LB_strain_avg, how='left', on='Strain')
LB_ctrl_merged_df["Control AUC_x"] = LB_ctrl_merged_df["Control AUC_y"]
LB_ctrl_merged_df["Control Endpoint OD_x"] = LB_ctrl_merged_df["Control Endpoint OD_y"]
LHR_LB_ctrl_avg = LB_ctrl_merged_df.rename(columns={"Control AUC_x" : "Avg Control AUC", "Control Endpoint OD_x" : "Avg Control Endpoint OD"})
LHR_LB_ctrl_avg = LHR_LB_ctrl_avg.drop(columns = ["Control AUC_y", "Control Endpoint OD_y"])
LHR_LB_ctrl_avg


In [ ]:
#get the average for each phage,strain replicate AUC and endpoint OD
LB_phage_avg = LHR_LB.groupby(["Phage", "Strain"])["Phage AUC", "Phage Endpoint OD"].agg(['mean', np.std, 'count']).reset_index()
LB_phage_avg.columns = ["Phage", "Strain", "Phage AUC", "Phage AUC StDev", "AUC count", "Phage Endpoint OD", "Phage OD StDev", "Phage Replicates"]

LB_phage_avg=LB_phage_avg.drop(columns=["AUC count"])

#substitute in average control AUCs and endpoint ODs for all strains
LB_phage_merged_df = pd.merge(LHR_LB_ctrl_avg, LB_phage_avg, how='left', on=['Phage', 'Strain'])
LB_phage_merged_df["Phage AUC_x"] = LB_phage_merged_df["Phage AUC_y"]
LB_phage_merged_df["Phage Endpoint OD_x"] = LB_phage_merged_df["Phage Endpoint OD_y"]
LHR_LB_avg = LB_phage_merged_df.rename(columns={"Phage AUC_x" : "Avg Phage AUC", "Phage Endpoint OD_x" : "Avg Phage Endpoint OD"})
LHR_LB_avg = LHR_LB_avg.drop(columns = ["Phage AUC_y", "Phage Endpoint OD_y"])

#dropping columns that will not be accurate for replicates
LHR_LB_avg = LHR_LB_avg.drop(columns = ["Phage ODs", "Control ODs", "Well"])

#calculate new AUC ratio based on average AUCs
LHR_LB_avg["AUC Ratio"] = LHR_LB_avg["Avg Phage AUC"] / LHR_LB_avg["Avg Control AUC"]

#calculate the AUC Ratio StDev
LHR_LB_avg["AUC Ratio StDev"] = LHR_LB_avg["AUC Ratio"] * np.sqrt((LHR_LB_avg["Phage AUC StDev"] / LHR_LB_avg["Avg Phage AUC"])**2 +
                                                                  (LHR_LB_avg["Control AUC StDev"] / LHR_LB_avg["Avg Control AUC"])**2)

#calculate new delta endpoint ODs based on average endpoint ODs
LHR_LB_avg["Delta Endpoint OD"] = LHR_LB_avg["Avg Control Endpoint OD"] - LHR_LB_avg["Avg Phage Endpoint OD"]

#calculate delta endpoint OD StDev
LHR_LB_avg["Delta Endpoint OD StDev"] = np.sqrt(LHR_LB_avg["Control OD StDev"]**2 + LHR_LB_avg["Phage OD StDev"]**2)

#remove duplicate rows
LHR_LB_avg = LHR_LB_avg.drop_duplicates()

#convert to a .csv
LHR_LB_avg.to_csv(#redacted, index=False)
print(len(LHR_LB_avg["Strain"].unique()))

#### Urine Average Replicates

In [ ]:
#Average the blank strain AUC and endpoint OD
urine_blanks_df = LHR_urine[LHR_urine["Phage"] == "blank_1"]
urine_strain_avg = urine_blanks_df.groupby('Strain')["Control AUC", "Control Endpoint OD"].agg(['mean', np.std, 'count']).reset_index()
urine_strain_avg.columns = ["Strain", "Control AUC", "Control AUC StDev", "AUC count", "Control Endpoint OD", "Control OD StDev", "Control Replicates"]
urine_strain_avg = urine_strain_avg.drop(columns=["AUC count"])
                   
# #get which strains didn't reach endpoint OD threshold (usually #redacted)
urine_strains_no_grow = urine_blanks_df[(urine_blanks_df["Strain"] != "urine") & (urine_blanks_df['Control Endpoint OD'] < strain_growth_threshold)]

# #substitute in average control AUCs and endpoint ODs for all strains
urine_ctrl_merged_df = pd.merge(LHR_urine, urine_strain_avg, how='left', on='Strain')
urine_ctrl_merged_df["Control AUC_x"] = urine_ctrl_merged_df["Control AUC_y"]
urine_ctrl_merged_df["Control Endpoint OD_x"] = urine_ctrl_merged_df["Control Endpoint OD_y"]
LHR_urine_ctrl_avg = urine_ctrl_merged_df.rename(columns={"Control AUC_x" : "Avg Control AUC", "Control Endpoint OD_x" : "Avg Control Endpoint OD"})
LHR_urine_ctrl_avg = LHR_urine_ctrl_avg.drop(columns = ["Control AUC_y", "Control Endpoint OD_y"])


In [ ]:
#get the average for each phage,strain replicate AUC and endpoint OD
urine_phage_avg = LHR_urine.groupby(["Phage", "Strain"])["Phage AUC", "Phage Endpoint OD"].agg(['mean', np.std, 'count']).reset_index()
urine_phage_avg.columns = ["Phage", "Strain", "Phage AUC", "Phage AUC StDev", "AUC count", "Phage Endpoint OD", "Phage OD StDev", "Phage Replicates"]

urine_phage_avg=urine_phage_avg.drop(columns=["AUC count"])

#substitute in average control AUCs and endpoint ODs for all strains
urine_phage_merged_df = pd.merge(LHR_urine_ctrl_avg, urine_phage_avg, how='left', on=['Phage', 'Strain'])
urine_phage_merged_df["Phage AUC_x"] = urine_phage_merged_df["Phage AUC_y"]
urine_phage_merged_df["Phage Endpoint OD_x"] = urine_phage_merged_df["Phage Endpoint OD_y"]
LHR_urine_avg = urine_phage_merged_df.rename(columns={"Phage AUC_x" : "Avg Phage AUC", "Phage Endpoint OD_x" : "Avg Phage Endpoint OD"})
LHR_urine_avg = LHR_urine_avg.drop(columns = ["Phage AUC_y", "Phage Endpoint OD_y"])

#dropping columns that will not be accurate for replicates
LHR_urine_avg = LHR_urine_avg.drop(columns = ["Phage ODs", "Control ODs", "Well"])

#calculate new AUC ratio based on average AUCs
LHR_urine_avg["AUC Ratio"] = LHR_urine_avg["Avg Phage AUC"] / LHR_urine_avg["Avg Control AUC"]

#calculate the AUC Ratio StDev
LHR_urine_avg["AUC Ratio StDev"] = LHR_urine_avg["AUC Ratio"] * np.sqrt((LHR_urine_avg["Phage AUC StDev"] / LHR_urine_avg["Avg Phage AUC"])**2 +
                                                                  (LHR_urine_avg["Control AUC StDev"] / LHR_urine_avg["Avg Control AUC"])**2)

#calculate new delta endpoint ODs based on average endpoint ODs
LHR_urine_avg["Delta Endpoint OD"] = LHR_urine_avg["Avg Control Endpoint OD"] - LHR_urine_avg["Avg Phage Endpoint OD"]

#calculate delta endpoint OD StDev
LHR_urine_avg["Delta Endpoint OD StDev"] = np.sqrt(LHR_urine_avg["Control OD StDev"]**2 + LHR_urine_avg["Phage OD StDev"]**2)

#remove duplicate rows
LHR_urine_avg = LHR_urine_avg.drop_duplicates()

#convert to a .csv
LHR_urine_avg.to_csv(#redacted, index=False)

#### Define Functions

In [ ]:
strain_growth_threshold = 1 #redacted
AUC_hit_cutoff = 1 #redacted

In [ ]:
#sum strains that aren't media
def sum_strains_NoMedia(group):
    return sum((group["Strain"] != "LB") & (group["Strain"] != "Urine"))

#sum strains that aren't media and ctrl OD > #redacted
def sum_strains_NoMedia_NoLowOD(group):
    return sum((group["Strain"] != "LB") & (group["Strain"] != "Urine") & (group["Avg Control Endpoint OD"] > strain_growth_threshold))

#sum strains hit (not media)
def sum_strains_hit_NoMedia(group):
    return sum((group["AUC Ratio"] < AUC_hit_cutoff) & (group["Strain"] != "LB") & (group["Strain"] != "Urine"))

#strain hit function ensuring control OD > #redacted
def sum_strains_hit_NoMedia_NoLowOD(group):
    return sum((group["AUC Ratio"] < AUC_hit_cutoff) & (group["Strain"] != "LB") & (group["Strain"] != "Urine") & (group["Avg Control Endpoint OD"] > strain_growth_threshold))

def get_strains_hit_percent(group):
    strains_hit = sum_strains_hit_NoMedia_NoLowOD(group)
    total_strains = sum_strains_NoMedia_NoLowOD(group)
    return strains_hit/total_strains * 100

def strains_hit_percent_auc_1std_high(group):
    strains_hit_AUC_high = sum(((group["AUC Ratio"] + group["AUC Ratio StDev"]) < AUC_hit_cutoff) & (group["Strain"] != "LB") & (group["Strain"] != "Urine") & 
                           (group["Avg Control Endpoint OD"] > strain_growth_threshold))
    total_strains = sum_strains_NoMedia_NoLowOD(group)
    return strains_hit_AUC_high/total_strains * 100

def strains_hit_percent_auc_1std_low(group):
    strains_hit_AUC_low = sum(((group["AUC Ratio"] - group["AUC Ratio StDev"]) < AUC_hit_cutoff) & (group["Strain"] != "LB") & (group["Strain"] != "Urine") & 
                           (group["Avg Control Endpoint OD"] > strain_growth_threshold))
    total_strains = sum_strains_NoMedia_NoLowOD(group)
    return strains_hit_AUC_low/total_strains * 100

def percent_DeltaEndpointOD_greater(group):
    num_strains_greater = sum(
        (group["Delta Endpoint OD"] > 1) & (group["Strain"] != "LB") & (group["Strain"] != "Urine") & (group["Avg Control Endpoint OD"] > strain_growth_threshold)
    )
    total_strains = sum_strains_NoMedia_NoLowOD(group)
    return num_strains_greater/total_strains * 100

def percent_DeltaEndOD_greater_stdHigh(group):
    num_strains_greater_stdHigh = sum(((group["Delta Endpoint OD"] + group["Delta Endpoint OD StDev"]) > 1) & (group["Strain"] != "LB") & (group["Strain"] != "Urine") &
                                (group["Avg Control Endpoint OD"] > strain_growth_threshold))
    total_strains = sum_strains_NoMedia_NoLowOD(group)
    return num_strains_greater_stdHigh/total_strains * 100

def percent_DeltaEndOD_greater_stdLow(group):
    num_strains_greater_stdLow = sum(((group["Delta Endpoint OD"] - group["Delta Endpoint OD StDev"]) > 1) & (group["Strain"] != "LB") & (group["Strain"] != "Urine") &
                                (group["Avg Control Endpoint OD"] > strain_growth_threshold))
    total_strains = sum_strains_NoMedia_NoLowOD(group)
    return num_strains_greater_stdLow/total_strains * 100


#### Get LB Summary Dataframe

In [ ]:
LB_LHR_summary = LHR_LB_avg.groupby("Phage", sort=False).apply(sum_strains_NoMedia).reset_index(name="Total Strains (#)")
LB_strains_noLowOD = LHR_LB_avg.groupby("Phage", sort=False).apply(sum_strains_NoMedia_NoLowOD)
LB_LHR_summary["Strains Ctrl OD > 1 (#)"] = LB_strains_noLowOD.values
LB_LHR_summary["Endpoint OD < 1 (%)"] = ((LB_LHR_summary["Total Strains (#)"] - LB_LHR_summary["Strains Ctrl OD > 1 (#)"]) / (LB_LHR_summary["Total Strains (#)"]))
LB_strains_hit_percent = LHR_LB_avg.groupby("Phage", sort=False).apply(get_strains_hit_percent)
LB_LHR_summary["Strains Hit (%)"] = LB_strains_hit_percent.values
LB_median_AUCs = LHR_LB_avg.groupby("Phage", sort=False)["AUC Ratio"].median()
LB_LHR_summary["Median AUC Ratio"] = LB_median_AUCs.values
LB_DeltaEndpointOD_greaterPercent = LHR_LB_avg.groupby("Phage", sort=False).apply(percent_DeltaEndpointOD_greater) 
LB_LHR_summary["Delta Endpoint OD > 1 (%)"] = LB_DeltaEndpointOD_greaterPercent.values
LB_median_DeltaEndpointOD = LHR_LB_avg.groupby("Phage", sort=False)["Delta Endpoint OD"].median()
LB_LHR_summary["Median Delta Endpoint OD"] = LB_median_DeltaEndpointOD.values
LB_max_titers = LHR_LB_avg.groupby("Phage", sort=False)["Titer"].max()
LB_LHR_summary["Titer"] = LB_max_titers.values
LB_LHR_summary["Media"] = "LB"

#high and low percentages for strain hits
LB_strains_hit_percent_auc1stdhigh = LHR_LB_avg.groupby("Phage", sort=False).apply(strains_hit_percent_auc_1std_high)
LB_strains_hit_percent_auc1stdlow = LHR_LB_avg.groupby("Phage", sort=False).apply(strains_hit_percent_auc_1std_low)
LB_LHR_summary["Strains Hit AUC 1 StDev High (%)"] = LB_strains_hit_percent_auc1stdhigh.values
LB_LHR_summary["Strains Hit AUC 1 StDev Low (%)"] = LB_strains_hit_percent_auc1stdlow.values

#high and low percentages for delta endpoint ODs
LB_DeltaEndOD_greaterPercent_1stdHigh = LHR_LB_avg.groupby("Phage", sort=False).apply(percent_DeltaEndOD_greater_stdHigh)
LB_DeltaEndOD_greaterPercent_1stdLow = LHR_LB_avg.groupby("Phage", sort=False).apply(percent_DeltaEndOD_greater_stdLow)
LB_LHR_summary["Delta End OD > 1, 1 StDev High (%)"] = LB_DeltaEndOD_greaterPercent_1stdHigh.values
LB_LHR_summary["Delta End OD > 1, 1 StDev Low (%)"] = LB_DeltaEndOD_greaterPercent_1stdLow.values

desired_column_order = [
    "Phage", "Strains Hit (%)", "Strains Hit AUC 1 StDev High (%)", "Strains Hit AUC 1 StDev Low (%)", "Median AUC Ratio", 
    "Delta Endpoint OD > 1 (%)", "Delta End OD > 1, 1 StDev High (%)", "Delta End OD > 1, 1 StDev Low (%)", "Median Delta Endpoint OD",
    "Total Strains (#)", "Strains Ctrl OD > 1 (#)", "Endpoint OD < 1 (%)", "Titer", "Media", 
]

LB_LHR_summary = LB_LHR_summary[desired_column_order]

LB_LHR_summary.to_csv(#redacted, index=False)

#### Get Urine Summary Dataframe

In [ ]:
urine_LHR_summary = LHR_urine_avg.groupby("Phage", sort=False).apply(sum_strains_NoMedia).reset_index(name="Total Strains (#)")
urine_strains_noLowOD = LHR_urine_avg.groupby("Phage", sort=False).apply(sum_strains_NoMedia_NoLowOD)
urine_LHR_summary["Strains Ctrl OD > 1 (#)"] = urine_strains_noLowOD.values
urine_LHR_summary["Endpoint OD <1 (%)"] = ((urine_LHR_summary["Total Strains (#)"] - urine_LHR_summary["Strains Ctrl OD > 1 (#)"]) / (urine_LHR_summary["Total Strains (#)"]))
urine_strains_hit_percent = LHR_urine_avg.groupby("Phage", sort=False).apply(get_strains_hit_percent)
urine_LHR_summary["Strains Hit (%)"] = urine_strains_hit_percent.values
urine_median_AUCs = LHR_urine_avg.groupby("Phage", sort=False)["AUC Ratio"].median()
urine_LHR_summary["Median AUC Ratio"] = urine_median_AUCs.values
urine_DeltaEndpointOD_greaterPercent = LHR_urine_avg.groupby("Phage", sort=False).apply(percent_DeltaEndpointOD_greater) 
urine_LHR_summary["Delta Endpoint OD > 1 (%)"] = urine_DeltaEndpointOD_greaterPercent.values
urine_median_DeltaEndpointOD = LHR_urine_avg.groupby("Phage", sort=False)["Delta Endpoint OD"].median()
urine_LHR_summary["Median Delta Endpoint OD"] = urine_median_DeltaEndpointOD.values
urine_max_titers = LHR_urine_avg.groupby("Phage", sort=False)["Titer"].max()
urine_LHR_summary["Titer"] = urine_max_titers.values
urine_LHR_summary["Media"] = "urine"

#high and low percentages for strain hits
urine_strains_hit_percent_auc1stdhigh = LHR_urine_avg.groupby("Phage", sort=False).apply(strains_hit_percent_auc_1std_high)
urine_strains_hit_percent_auc1stdlow = LHR_urine_avg.groupby("Phage", sort=False).apply(strains_hit_percent_auc_1std_low)
urine_LHR_summary["Strains Hit AUC 1 StDev High (%)"] = urine_strains_hit_percent_auc1stdhigh.values
urine_LHR_summary["Strains Hit AUC 1 StDev Low (%)"] = urine_strains_hit_percent_auc1stdlow.values

#high and low percentages for delta endpoint ODs
urine_DeltaEndOD_greaterPercent_1stdHigh = LHR_urine_avg.groupby("Phage", sort=False).apply(percent_DeltaEndOD_greater_stdHigh)
urine_DeltaEndOD_greaterPercent_1stdLow = LHR_urine_avg.groupby("Phage", sort=False).apply(percent_DeltaEndOD_greater_stdLow)
urine_LHR_summary["Delta End OD > 1, 1 StDev High (%)"] = urine_DeltaEndOD_greaterPercent_1stdHigh.values
urine_LHR_summary["Delta End OD > 1, 1 StDev Low (%)"] = urine_DeltaEndOD_greaterPercent_1stdLow.values

desired_column_order = [
    "Phage", "Strains Hit (%)", "Strains Hit AUC 1 StDev High (%)", "Strains Hit AUC 1 StDev Low (%)", "Median AUC Ratio", 
    "Delta Endpoint OD > 1 (%)", "Delta End OD > 1, 1 StDev High (%)", "Delta End OD > 1, 1 StDev Low (%)", "Median Delta Endpoint OD",
    "Total Strains (#)", "Strains Ctrl OD > 1 (#)", "Endpoint OD < 1 (%)", "Titer", "Media", 
]

urine_LHR_summary = urine_LHR_summary[desired_column_order]

urine_LHR_summary.to_csv(#redacted, index=False)

## Genetically Unique Isolates Below

In [ ]:
unique_isolates = pd.read_csv(#redacted, header=None, names=["Strain", "Patient"])
unique_isolates=unique_isolates.drop(columns = ["Patient"])
LHR_LB_uniqueIsolates = pd.merge(unique_isolates, LHR_LB_avg, on="Strain", how="left")
LHR_urine_uniqueIsolates = pd.merge(unique_isolates, LHR_urine_avg, on="Strain", how="left")
LHR_LB_uniqueIsolates.to_csv(#redacted)
LHR_urine_uniqueIsolates.to_csv(#redacted)

In [ ]:
LB_LHR_summary_uniqueIsolates = LHR_LB_uniqueIsolates.groupby("Phage", sort=False).apply(sum_strains_NoMedia).reset_index(name="Total Strains (#)")
LB_strains_noLowOD = LHR_LB_uniqueIsolates.groupby("Phage", sort=False).apply(sum_strains_NoMedia_NoLowOD)
LB_LHR_summary_uniqueIsolates["Strains Ctrl OD > 1 (#)"] = LB_strains_noLowOD.values
LB_LHR_summary_uniqueIsolates["Endpoint OD < 1 (%)"] = ((LB_LHR_summary_uniqueIsolates["Total Strains (#)"] - LB_LHR_summary_uniqueIsolates["Strains Ctrl OD > 1 (#)"]) / (LB_LHR_summary_uniqueIsolates["Total Strains (#)"]))
LB_strains_hit_percent = LHR_LB_uniqueIsolates.groupby("Phage", sort=False).apply(get_strains_hit_percent)
LB_LHR_summary_uniqueIsolates["Strains Hit (%)"] = LB_strains_hit_percent.values
LB_median_AUCs = LHR_LB_uniqueIsolates.groupby("Phage", sort=False)["AUC Ratio"].median()
LB_LHR_summary_uniqueIsolates["Median AUC Ratio"] = LB_median_AUCs.values
LB_DeltaEndpointOD_greaterPercent = LHR_LB_uniqueIsolates.groupby("Phage", sort=False).apply(percent_DeltaEndpointOD_greater) 
LB_LHR_summary_uniqueIsolates["Delta Endpoint OD > 1 (%)"] = LB_DeltaEndpointOD_greaterPercent.values
LB_median_DeltaEndpointOD = LHR_LB_uniqueIsolates.groupby("Phage", sort=False)["Delta Endpoint OD"].median()
LB_LHR_summary_uniqueIsolates["Median Delta Endpoint OD"] = LB_median_DeltaEndpointOD.values
LB_max_titers = LHR_LB_uniqueIsolates.groupby("Phage", sort=False)["Titer"].max()
LB_LHR_summary_uniqueIsolates["Titer"] = LB_max_titers.values
LB_LHR_summary_uniqueIsolates["Media"] = "LB"

#high and low percentages for strain hits
LB_strains_hit_percent_auc1stdhigh = LHR_LB_uniqueIsolates.groupby("Phage", sort=False).apply(strains_hit_percent_auc_1std_high)
LB_strains_hit_percent_auc1stdlow = LHR_LB_uniqueIsolates.groupby("Phage", sort=False).apply(strains_hit_percent_auc_1std_low)
LB_LHR_summary_uniqueIsolates["Strains Hit AUC 1 StDev High (%)"] = LB_strains_hit_percent_auc1stdhigh.values
LB_LHR_summary_uniqueIsolates["Strains Hit AUC 1 StDev Low (%)"] = LB_strains_hit_percent_auc1stdlow.values

#high and low percentages for delta endpoint ODs
LB_DeltaEndOD_greaterPercent_1stdHigh = LHR_LB_uniqueIsolates.groupby("Phage", sort=False).apply(percent_DeltaEndOD_greater_stdHigh)
LB_DeltaEndOD_greaterPercent_1stdLow = LHR_LB_uniqueIsolates.groupby("Phage", sort=False).apply(percent_DeltaEndOD_greater_stdLow)
LB_LHR_summary_uniqueIsolates["Delta End OD > 0.2, 1 StDev High (%)"] = LB_DeltaEndOD_greaterPercent_1stdHigh.values
LB_LHR_summary_uniqueIsolates["Delta End OD > 0.2, 1 StDev Low (%)"] = LB_DeltaEndOD_greaterPercent_1stdLow.values

desired_column_order = [
    "Phage", "Strains Hit (%)", "Strains Hit AUC 1 StDev High (%)", "Strains Hit AUC 1 StDev Low (%)", "Median AUC Ratio", 
    "Delta Endpoint OD > 1 (%)", "Delta End OD > 1, 1 StDev High (%)", "Delta End OD > 1, 1 StDev Low (%)", "Median Delta Endpoint OD",
    "Total Strains (#)", "Strains Ctrl OD > 1 (#)", "Endpoint OD < 1 (%)", "Titer", "Media", 
]

LB_LHR_summary_uniqueIsolates = LB_LHR_summary_uniqueIsolates[desired_column_order]

LB_LHR_summary_uniqueIsolates.to_csv(#redacted, index=False)

In [ ]:
urine_LHR_summary_uniqueIsolates = LHR_urine_uniqueIsolates.groupby("Phage", sort=False).apply(sum_strains_NoMedia).reset_index(name="Total Strains (#)")
urine_strains_noLowOD = LHR_urine_uniqueIsolates.groupby("Phage", sort=False).apply(sum_strains_NoMedia_NoLowOD)
urine_LHR_summary_uniqueIsolates["Strains Ctrl OD > 1 (#)"] = urine_strains_noLowOD.values
urine_LHR_summary_uniqueIsolates["Endpoint OD < 1 (%)"] = ((urine_LHR_summary_uniqueIsolates["Total Strains (#)"] - urine_LHR_summary_uniqueIsolates["Strains Ctrl OD > 1 (#)"]) / (urine_LHR_summary_uniqueIsolates["Total Strains (#)"]))
urine_strains_hit_percent = LHR_urine_uniqueIsolates.groupby("Phage", sort=False).apply(get_strains_hit_percent)
urine_LHR_summary_uniqueIsolates["Strains Hit (%)"] = urine_strains_hit_percent.values
urine_median_AUCs = LHR_urine_uniqueIsolates.groupby("Phage", sort=False)["AUC Ratio"].median()
urine_LHR_summary_uniqueIsolates["Median AUC Ratio"] = urine_median_AUCs.values
urine_DeltaEndpointOD_greaterPercent = LHR_urine_uniqueIsolates.groupby("Phage", sort=False).apply(percent_DeltaEndpointOD_greater) 
urine_LHR_summary_uniqueIsolates["Delta Endpoint OD > 1 (%)"] = urine_DeltaEndpointOD_greaterPercent.values
urine_median_DeltaEndpointOD = LHR_urine_uniqueIsolates.groupby("Phage", sort=False)["Delta Endpoint OD"].median()
urine_LHR_summary_uniqueIsolates["Median Delta Endpoint OD"] = urine_median_DeltaEndpointOD.values
urine_max_titers = LHR_urine_uniqueIsolates.groupby("Phage", sort=False)["Titer"].max()
urine_LHR_summary_uniqueIsolates["Titer"] = urine_max_titers.values
urine_LHR_summary_uniqueIsolates["Media"] = "urine"

#high and low percentages for strain hits
urine_strains_hit_percent_auc1stdhigh = LHR_urine_uniqueIsolates.groupby("Phage", sort=False).apply(strains_hit_percent_auc_1std_high)
urine_strains_hit_percent_auc1stdlow = LHR_urine_uniqueIsolates.groupby("Phage", sort=False).apply(strains_hit_percent_auc_1std_low)
urine_LHR_summary_uniqueIsolates["Strains Hit AUC 1 StDev High (%)"] = urine_strains_hit_percent_auc1stdhigh.values
urine_LHR_summary_uniqueIsolates["Strains Hit AUC 1 StDev Low (%)"] = urine_strains_hit_percent_auc1stdlow.values

#high and low percentages for delta endpoint ODs
urine_DeltaEndOD_greaterPercent_1stdHigh = LHR_urine_uniqueIsolates.groupby("Phage", sort=False).apply(percent_DeltaEndOD_greater_stdHigh)
urine_DeltaEndOD_greaterPercent_1stdLow = LHR_urine_uniqueIsolates.groupby("Phage", sort=False).apply(percent_DeltaEndOD_greater_stdLow)
urine_LHR_summary_uniqueIsolates["Delta End OD > 1, 1 StDev High (%)"] = urine_DeltaEndOD_greaterPercent_1stdHigh.values
urine_LHR_summary_uniqueIsolates["Delta End OD > 1, 1 StDev Low (%)"] = urine_DeltaEndOD_greaterPercent_1stdLow.values

desired_column_order = [
    "Phage", "Strains Hit (%)", "Strains Hit AUC 1 StDev High (%)", "Strains Hit AUC 1 StDev Low (%)", "Median AUC Ratio", 
    "Delta Endpoint OD > 1 (%)", "Delta End OD > 1, 1 StDev High (%)", "Delta End OD > 1, 1 StDev Low (%)", "Median Delta Endpoint OD",
    "Total Strains (#)", "Strains Ctrl OD > 1 (#)", "Endpoint OD < 1 (%)", "Titer", "Media", 
]

urine_LHR_summary_uniqueIsolates = urine_LHR_summary_uniqueIsolates[desired_column_order]

urine_LHR_summary_uniqueIsolates.to_csv(#redacted, index=False)

In [ ]:
###### cnum sheet manipulation to subset to the #redacted isolates tested
duplicate_cnum = cnum_clinic[cnum_clinic.duplicated('Name..')]

cnum_clinic_nona = cnum_clinic.dropna(subset=['Name..'])

six_prepause_isolates = ['c000675', 'c000678', 'c000681', 'c000686', 'c000687', 'c000692']

cnum_clinic_subset = cnum_clinic[(cnum_clinic['specimen_collection_datetime'].str.startswith('2023')) | (cnum_clinic['Name..'].isin(six_prepause_isolates))]
cnum_clinic_subset = cnum_clinic_subset.iloc[:,:-31]
#print(len(cnum_clinic_subset))
#print((cnum_clinic_subset['specimen_collection_time_description'] == 'Pre First Dose').sum())
#print(cnum_clinic_subset['specimen_collection_time_description'].unique())
baseline_isolates = cnum_clinic_subset[(cnum_clinic_subset['visit'] == 1) & ((cnum_clinic_subset['specimen_collection_time_description'] == 'Pre-Dose') |
                                                                           (cnum_clinic_subset['specimen_collection_time_description'] == 'Pre First Dose'))]

#c00857 will be the effective baseline isolate for patient 103009 since a true baseline was not collected
baseline_isolates = pd.concat([baseline_isolates, cnum_clinic_subset[cnum_clinic_subset["Name.."] == "C000857"]])

print(baseline_isolates.columns.unique())

#Get those baseline isolates into a nice csv
baseline_isolate_columns = ["subject_id", "Name.."]
baseline_isolates = baseline_isolates[baseline_isolate_columns]
baseline_isolates = baseline_isolates.rename(columns={"subject_id" : "Patient", "Name.." : "Strain"})
baseline_isolates = baseline_isolates[["Strain", "Patient"]]
baseline_isolates["Strain"] = baseline_isolates["Strain"].str.upper()
baseline_isolates.to_csv(#redacted, index=False)
print(len(baseline_isolates.Strain.unique()))
print(len(baseline_isolates.Patient.unique()))

In [ ]:
#get the clinical isolate cnum subset as a .csv
LHR_unique = LHR_LB_avg.Strain.unique()

cnum_unique = list(cnum_clinic_subset['Name..'].unique())
cnum_unique = str(cnum_unique).upper()
#print(cnum_unique)

missing_cnums = [x for x in LHR_unique if x not in cnum_unique]
print(missing_cnums)
#print(len(cnum_clinic_subset))
#cnum_clinic_subset.to_csv("cnum_subset.csv", index=False)

#C000845 and C000836 are not in the cnum dataframe, adding manually to confirmed correct locations
cnum_subset_updated = pd.read_csv(#redacted)
cnum_unique_updated = list(cnum_subset_updated['Name..'].unique())
cnum_unique_updated = str(cnum_unique_updated).upper()
print(cnum_unique_updated)
missing_cnums = [x for x in LHR_unique if x not in cnum_unique_updated]
print(missing_cnums)


### Baseline Isolate LHR Analysis Below

In [ ]:
LHR_LB_baselineIsolates = pd.merge(baseline_isolate_strains, LHR_LB_avg, on="Strain", how="left")
LHR_urine_baselineIsolates = pd.merge(baseline_isolate_strains, LHR_urine_avg, on="Strain", how="left")
LHR_LB_baselineIsolates.to_csv(#redacted)
LHR_urine_baselineIsolates.to_csv(#redacted)

In [ ]:
no_grow_urine = LHR_urine_baselineIsolates[LHR_urine_baselineIsolates["Avg Control Endpoint OD"] < 1]
print(no_grow_urine.Strain.unique())

In [ ]:
LB_LHR_summary_baselineIsolates = LHR_LB_baselineIsolates.groupby("Phage", sort=False).apply(sum_strains_NoMedia).reset_index(name="Total Strains (#)")
LB_strains_noLowOD = LHR_LB_baselineIsolates.groupby("Phage", sort=False).apply(sum_strains_NoMedia_NoLowOD)
LB_LHR_summary_baselineIsolates["Strains Ctrl OD > 1 (#)"] = LB_strains_noLowOD.values
LB_LHR_summary_baselineIsolates["Endpoint OD < 1 (%)"] = ((LB_LHR_summary_baselineIsolates["Total Strains (#)"] - LB_LHR_summary_baselineIsolates["Strains Ctrl OD > 1 (#)"]) / (LB_LHR_summary_baselineIsolates["Total Strains (#)"]))
LB_strains_hit_percent = LHR_LB_baselineIsolates.groupby("Phage", sort=False).apply(get_strains_hit_percent)
LB_LHR_summary_baselineIsolates["Strains Hit (%)"] = LB_strains_hit_percent.values
LB_median_AUCs = LHR_LB_baselineIsolates.groupby("Phage", sort=False)["AUC Ratio"].median()
LB_LHR_summary_baselineIsolates["Median AUC Ratio"] = LB_median_AUCs.values
LB_DeltaEndpointOD_greaterPercent = LHR_LB_baselineIsolates.groupby("Phage", sort=False).apply(percent_DeltaEndpointOD_greater) 
LB_LHR_summary_baselineIsolates["Delta Endpoint OD > 1 (%)"] = LB_DeltaEndpointOD_greaterPercent.values
LB_median_DeltaEndpointOD = LHR_LB_baselineIsolates.groupby("Phage", sort=False)["Delta Endpoint OD"].median()
LB_LHR_summary_baselineIsolates["Median Delta Endpoint OD"] = LB_median_DeltaEndpointOD.values
LB_max_titers = LHR_LB_baselineIsolates.groupby("Phage", sort=False)["Titer"].max()
LB_LHR_summary_baselineIsolates["Titer"] = LB_max_titers.values
LB_LHR_summary_baselineIsolates["Media"] = "LB"

#high and low percentages for strain hits
LB_strains_hit_percent_auc1stdhigh = LHR_LB_baselineIsolates.groupby("Phage", sort=False).apply(strains_hit_percent_auc_1std_high)
LB_strains_hit_percent_auc1stdlow = LHR_LB_baselineIsolates.groupby("Phage", sort=False).apply(strains_hit_percent_auc_1std_low)
LB_LHR_summary_baselineIsolates["Strains Hit AUC 1 StDev High (%)"] = LB_strains_hit_percent_auc1stdhigh.values
LB_LHR_summary_baselineIsolates["Strains Hit AUC 1 StDev Low (%)"] = LB_strains_hit_percent_auc1stdlow.values

#high and low percentages for delta endpoint ODs
LB_DeltaEndOD_greaterPercent_1stdHigh = LHR_LB_baselineIsolates.groupby("Phage", sort=False).apply(percent_DeltaEndOD_greater_stdHigh)
LB_DeltaEndOD_greaterPercent_1stdLow = LHR_LB_baselineIsolates.groupby("Phage", sort=False).apply(percent_DeltaEndOD_greater_stdLow)
LB_LHR_summary_baselineIsolates["Delta End OD > 0.2, 1 StDev High (%)"] = LB_DeltaEndOD_greaterPercent_1stdHigh.values
LB_LHR_summary_baselineIsolates["Delta End OD > 0.2, 1 StDev Low (%)"] = LB_DeltaEndOD_greaterPercent_1stdLow.values

desired_column_order = [
    "Phage", "Strains Hit (%)", "Strains Hit AUC 1 StDev High (%)", "Strains Hit AUC 1 StDev Low (%)", "Median AUC Ratio", 
    "Delta Endpoint OD > 1 (%)", "Delta End OD > 1, 1 StDev High (%)", "Delta End OD > 1, 1 StDev Low (%)", "Median Delta Endpoint OD",
    "Total Strains (#)", "Strains Ctrl OD > 1 (#)", "Endpoint OD < 1 (%)", "Titer", "Media", 
]

LB_LHR_summary_baselineIsolates = LB_LHR_summary_baselineIsolates[desired_column_order]

LB_LHR_summary_baselineIsolates.to_csv(#redacted, index=False)

In [ ]:
urine_LHR_summary_baselineIsolates = LHR_urine_baselineIsolates.groupby("Phage", sort=False).apply(sum_strains_NoMedia).reset_index(name="Total Strains (#)")
urine_strains_noLowOD = LHR_urine_baselineIsolates.groupby("Phage", sort=False).apply(sum_strains_NoMedia_NoLowOD)
urine_LHR_summary_baselineIsolates["Strains Ctrl OD > 1 (#)"] = urine_strains_noLowOD.values
urine_LHR_summary_baselineIsolates["Endpoint OD < 1 (%)"] = ((urine_LHR_summary_baselineIsolates["Total Strains (#)"] - urine_LHR_summary_baselineIsolates["Strains Ctrl OD > 1 (#)"]) / (urine_LHR_summary_baselineIsolates["Total Strains (#)"]))
urine_strains_hit_percent = LHR_urine_baselineIsolates.groupby("Phage", sort=False).apply(get_strains_hit_percent)
urine_LHR_summary_baselineIsolates["Strains Hit (%)"] = urine_strains_hit_percent.values
urine_median_AUCs = LHR_urine_baselineIsolates.groupby("Phage", sort=False)["AUC Ratio"].median()
urine_LHR_summary_baselineIsolates["Median AUC Ratio"] = urine_median_AUCs.values
urine_DeltaEndpointOD_greaterPercent = LHR_urine_baselineIsolates.groupby("Phage", sort=False).apply(percent_DeltaEndpointOD_greater) 
urine_LHR_summary_baselineIsolates["Delta Endpoint OD > 1 (%)"] = urine_DeltaEndpointOD_greaterPercent.values
urine_median_DeltaEndpointOD = LHR_urine_baselineIsolates.groupby("Phage", sort=False)["Delta Endpoint OD"].median()
urine_LHR_summary_baselineIsolates["Median Delta Endpoint OD"] = urine_median_DeltaEndpointOD.values
urine_max_titers = LHR_urine_baselineIsolates.groupby("Phage", sort=False)["Titer"].max()
urine_LHR_summary_baselineIsolates["Titer"] = urine_max_titers.values
urine_LHR_summary_baselineIsolates["Media"] = "urine"

#high and low percentages for strain hits
urine_strains_hit_percent_auc1stdhigh = LHR_urine_baselineIsolates.groupby("Phage", sort=False).apply(strains_hit_percent_auc_1std_high)
urine_strains_hit_percent_auc1stdlow = LHR_urine_baselineIsolates.groupby("Phage", sort=False).apply(strains_hit_percent_auc_1std_low)
urine_LHR_summary_baselineIsolates["Strains Hit AUC 1 StDev High (%)"] = urine_strains_hit_percent_auc1stdhigh.values
urine_LHR_summary_baselineIsolates["Strains Hit AUC 1 StDev Low (%)"] = urine_strains_hit_percent_auc1stdlow.values

#high and low percentages for delta endpoint ODs
urine_DeltaEndOD_greaterPercent_1stdHigh = LHR_urine_baselineIsolates.groupby("Phage", sort=False).apply(percent_DeltaEndOD_greater_stdHigh)
urine_DeltaEndOD_greaterPercent_1stdLow = LHR_urine_baselineIsolates.groupby("Phage", sort=False).apply(percent_DeltaEndOD_greater_stdLow)
urine_LHR_summary_baselineIsolates["Delta End OD > 0.2, 1 StDev High (%)"] = urine_DeltaEndOD_greaterPercent_1stdHigh.values
urine_LHR_summary_baselineIsolates["Delta End OD > 0.2, 1 StDev Low (%)"] = urine_DeltaEndOD_greaterPercent_1stdLow.values

desired_column_order = [
    "Phage", "Strains Hit (%)", "Strains Hit AUC 1 StDev High (%)", "Strains Hit AUC 1 StDev Low (%)", "Median AUC Ratio", 
    "Delta Endpoint OD > 1 (%)", "Delta End OD > 1, 1 StDev High (%)", "Delta End OD > 1, 1 StDev Low (%)", "Median Delta Endpoint OD",
    "Total Strains (#)", "Strains Ctrl OD > 1 (#)", "Endpoint OD < 1 (%)", "Titer", "Media", 
]

urine_LHR_summary_baselineIsolates = urine_LHR_summary_baselineIsolates[desired_column_order]

urine_LHR_summary_baselineIsolates.to_csv(#redacted, index=False)

In [ ]:
theBaddie = LHR_urine_baselineIsolates["Strain"][(LHR_urine_baselineIsolates["AUC Ratio"] > 1) & (LHR_urine_baselineIsolates["Phage"] == "CK618_bactrim") &
                                                (LHR_urine_baselineIsolates["Avg Control Endpoint OD"] > 1)]
print(theBaddie)

In [ ]:
theBaddies = theBaddie = LHR_urine_baselineIsolates["Strain"][(LHR_urine_baselineIsolates["AUC Ratio"] > 1) & (LHR_urine_baselineIsolates["Phage"] == "CK618_bactrim") &
                                                (LHR_urine_baselineIsolates["Avg Control Endpoint OD"] > 1)]
print(theBaddie)